In [ ]:
using QuantumCollocation

using LinearAlgebra
using Optim
using Random

using CairoMakie
using ForwardDiff

# Example 1.1
-----
**How to evaluate Newton's method**

In [ ]:
function h(x)
    return x.^4 + x.^3 - x.^2 - x
end

function ∇h(x)
    return 4.0*x.^3 + 3.0*x.^2 - 2.0*x - 1.0
end

function ∇²h(x)
    return 12.0*x.^2 + 6.0*x - 2.0
end

x = range(-1.75,1.25,1000)

In [ ]:
function newton_step(xᵢ)
    return xᵢ - ∇²h(xᵢ)\∇h(xᵢ)
end

## Initial guess
# xᵢ = 1.19
xᵢ = 0.0

## Initial plot
fig1 = Figure()
ax1 = Axis(fig1[1,1])
lines!(ax1, x, h(x))

In [ ]:
xᵢ₊₁ = newton_step(xᵢ) 
plot!(ax1, [xᵢ], [h(xᵢ)], color = :orange, marker = 'x', markersize=25)
xᵢ = xᵢ₊₁
fig1

In [ ]:
function regularized_newton_step(xᵢ)
    β = 1.0
    H = ∇²h(xᵢ)
    while !isposdef(H)
        H = H + β*I
    end
    return xᵢ - H\∇h(xᵢ)
end

## Initial guess
# xᵢ = 1.19
xᵢ = 0.0

## Initial plot
fig1 = Figure()
ax1 = Axis(fig1[1,1])
lines!(ax1, x, h(x))

In [ ]:
xᵢ₊₁ = regularized_newton_step(xᵢ) 
plot!(ax1, [xᵢ], [h(xᵢ)], color = :red, marker = 'x', markersize=25)
xᵢ = xᵢ₊₁
fig1

In [ ]:
function backtracking_regularized_newton_step(xᵢ)
    H = ∇²h(xᵢ)

    ## regularization
    β = 1.0
    while !isposdef(H)
        H = H + β*I
    end
    Δx = -H\∇h(xᵢ)

    ## line search
    b = 0.1
    c = 0.25
    α = 1.0
    while h(xᵢ + α*Δx) > h(xᵢ) + b*α*∇h(xᵢ)*Δx
        α = c*α
    end
    
    return xᵢ + α*Δx
end

## Initial guess
# xᵢ = 1.19
xᵢ = 0.0

## Initial plot
fig1 = Figure()
ax1 = Axis(fig1[1,1])
lines!(ax1, x, h(x))

In [ ]:
xᵢ₊₁ = backtracking_regularized_newton_step(xᵢ) 
plot!(ax1, [xᵢ], [h(xᵢ)], color = :green, marker = 'x', markersize=25)
xᵢ = xᵢ₊₁
fig1

# Example 1.2
-----
**How to set up and solve a GRAPE problem**

The GRAPE algorithm comes from NMR in 2004: https://doi.org/10.1016/j.jmr.2004.11.004

![GRAPE algorithm](images/gr1.gif)

A Julia version: https://github.com/JuliaQuantumControl/GRAPE.jl

Let's create a quantum system using Piccolo. `PAULIS` and `GATES` contain some helpful matrices. We pick a system with a little drift, just so we don't have an obvious solution.

In [ ]:
system = QuantumSystem(0.01 * PAULIS[:Z], [PAULIS[:X], PAULIS[:Y]])
U_goal = GATES[:X]

T = 25
Δt = 0.2

It is easiest to work with real-valued variables, especially when we want to use tools like automatic differentiation. Piccolo has a number of isomorphism utilities in `QuantumCollocation::Isomorophisms.jl`.

In [ ]:
Ũ⃗_goal = operator_to_iso_vec(U_goal)

It's not too hard to convert back.

In [ ]:
iso_vec_to_operator(Ũ⃗_goal)

We can use `QuantumCollocation::Rollouts.jl` to get a trajectory from the controls. Notice that we pass the controls and timesteps.

In [ ]:
Random.seed!(1234)

a_guess = randn(2, T)

## Rollout returns the unitary at each time step (as a vector)
Ũ⃗s = unitary_rollout(a_guess, fill(Δt, T), system)

Our random guess did not implement the X gate.

In [ ]:
## Plot the trajectory of σₓ
fig1 = Figure()
ax1_1 = fig1[1, 1] = Axis(fig1, xlabel = "Time", ylabel = "⟨X(t)⟩")
ax1_2 = fig1[2, 1] = Axis(fig1, xlabel = "Time", ylabel = "⟨Y(t)⟩")
ax1_3 = fig1[3, 1] = Axis(fig1, xlabel = "Time", ylabel = "⟨Z(t)⟩")

function plot_paulis(a)
    ψ_0 = [1; 0]

    x = Float64[]
    y = Float64[]
    z = Float64[]
    for Ũ⃗ ∈ eachcol(unitary_rollout(a, fill(Δt, T), system))
        U = Matrix{ComplexF64}(iso_vec_to_operator(Ũ⃗))
        ψ_t = U * ψ_0
        push!(x, real(ψ_t'PAULIS[:X]*ψ_t))
        push!(y, real(ψ_t'PAULIS[:Y]*ψ_t))
        push!(z, real(ψ_t'PAULIS[:Z]*ψ_t))
    end
    lines!(ax1_1, Δt * (1:T), x)
    lines!(ax1_2, Δt * (1:T), y)
    lines!(ax1_3, Δt * (1:T), z)
    return fig1
end

plot_paulis(a_guess)

`ForwardDiff.jl` allows us to compute the gradient of a scalar objective--in this case, infidelity.

In [ ]:
function f(x::AbstractVector)
    a = reshape(x, (2, T))
    Ũ⃗ = unitary_rollout(a, fill(Δt, T), system)
    return 1 - iso_vec_unitary_fidelity(Ũ⃗[:, end], Ũ⃗_goal)
end

∇f(x) = ForwardDiff.gradient(f, x)

In [ ]:
## Initial condition
xᵢ = copy(vec(a_guess));

In [ ]:
## Gradient descent

# Learning rate
λ = 0.25

# Step
xᵢ₊₁ = xᵢ - λ * ∇f(xᵢ)

f(xᵢ₊₁) |> println
xᵢ .= xᵢ₊₁
plot_paulis(reshape(xᵢ₊₁, (2, T)))

The controls seem to do the job! Let's compare the original and the optimized.

In [ ]:
function plot_control(a, title="")
    fig = Figure()

    ax1 = Axis(fig[1, 1], title="Initial", xlabel = "Time", ylabel = "Control")
    stairs!(ax, Δt * (1:T), a_guess[1, :])
    stairs!(ax, Δt * (1:T), a_guess[2, :])

    ax2 = fig[1, 2] = Axis(fig2, title=title, xlabel = "Time")
    stairs!(ax1, Δt * (1:T), a[1, :])
    stairs!(ax2, Δt * (1:T), a[2, :])
    return fig2
end

## Plot the optimized control
a_optimized = reshape(xᵢ₊₁, (2, T))
plot_control(a_optimized, "Gradient descent")

Of course, we don't need to code up our own gradient descent.

In [ ]:
res = optimize(f, ∇f, vec(a_guess), GradientDescent(), inplace=false)

In [ ]:
## Plot the optimized control
a_optimized = reshape(res.minimizer, (2, T))
plot_control(a_optimized, "GradientDescent (Optim.jl)")

# Exercises
-----

## Excercise 1.1
**Accelerate GRAPE using Newton's method**

Let's implement second-order GRAPE: https://doi.org/10.1016/j.jmr.2011.07.023

In [ ]:
## Compute the Hessian from f
# ∇²f(x) = # TODO
∇²f(x) = ForwardDiff.hessian(f, x)

function Δx(x)
   # TODO: Implement the Newton's step using backslash
   return -∇²f(x) \ ∇f(x)
end

In [ ]:
## Evaluate the Newton step
Δx(vec(a_guess))

The previous cell should remind us about the importance of regularization!

In [ ]:
function f_reg(x; λ=1e-8)
    # TODO: Regularize on norm(x) here 
    return f(x) + λ * x'x
end

function ∇f_reg!(G, x)
    # TODO: In-place gradient computation
    ForwardDiff.gradient!(G, f_reg, x)
end

function ∇²f_reg!(H, x)
    # TODO: In-place Hessian computation
    ForwardDiff.hessian!(H, f_reg, x)
end

function ∇²f_reg!(H, x)
    # TODO: in-place Hessian computation
    ForwardDiff.hessian!(H, f_reg, x)
end

function Δx_reg(x)
    G = zeros(length(x))
    H = zeros(length(x), length(x))
    ∇²f_reg!(H, x)
    ∇f_reg!(G, x)
    return -H \ G
 end

In [ ]:
Δx_reg(vec(a_guess))

Let's try to solve our regularized problem three ways using optim:

Gradient and Hessian:

1. Newton's method (Optim implements a line search for us!)
2. Newton's method with a trust region

Gradient only:

3. L-BFGS

In [ ]:
## Pass these options to optimize to limit the iterations and save function evaluations.
optim_options = Optim.Options(iterations=10, store_trace=true);

In [ ]:
## Newton's method
# res_newton = # TODO 

res_newton = optimize(
    f_reg, ∇f_reg!, ∇²f_reg!, vec(a_guess), Newton(),
    optim_options
)

In [ ]:
## Newton's method with a trust region
# res_newton_tr = # TODO 

res_newton_tr = optimize(
    f_reg, ∇f_reg!, ∇²f_reg!, vec(a_guess), NewtonTrustRegion(),
    optim_options
)

In [ ]:
## LBFGS
# res_lbfgs = # TODO

res_lbfgs = optimize(
    f_reg, ∇f_reg!, vec(a_guess), LBFGS(),
    optim_options
)

Compare the convergence rates.

In [ ]:
fig3 = Figure()
ax3 = fig3[1, 1] = Axis(fig3, xlabel="Iteration", ylabel="Objective", yscale=log10)

## Add a gradient descent for comparison
res_gd = optimize(f_reg, ∇f_reg!, vec(a_guess), GradientDescent(), optim_options)

lines!(ax3, Optim.f_trace(res_newton), label="Newton")
lines!(ax3, Optim.f_trace(res_newton_tr), label="NewtonTrustRegion")
lines!(ax3, Optim.f_trace(res_lbfgs), label="LBFGS")
lines!(ax3, Optim.f_trace(res_gd), label="GradientDescent")

Legend(fig3[1, 2], ax3, nbanks=1)
fig3

Everyone else does a lot better than Newton's method for our high dimensional problem.

## Excercise 1.2
**Add a function basis to GRAPE**

<img src="images/crab.jpg" alt="CRAB algorithm" style="width:800px;"/>

The previous control solutions seemed pretty wild. Can we do better? 
- One decade of CRAB: https://doi.org/10.1088/1361-6633/ac723c

What are some other good functions to use?
- Slepians provide bandwidth limits: https://doi.org/10.1103/PhysRevA.97.062346
- Splines minimize the curvature between points: https://github.com/LLNL/Juqbox.jl

The image is showing the idea of a _control landscape_. There are two versions, the landscape over the control parameters $c_1$ and $c_2$ (left) and the landscape over the final state (right). The point is that, while fidelity is a convex function with one maximum, control landscapes can be much more complicated.

In this example, we will pick a set of basis functions and expand the control in that basis,
\begin{equation}
    a(t) = b_0 + \sum_{j=1}^n \theta_j b_j(t).
\end{equation}
The optimizable parameters are now the coefficients in this basis. The idea here is _dimensionality reduction_ to simplify the search.

In [ ]:
## First n=5 entries in a Fourier series, including the constant term
n = 5
# fourier_series = # TODO 
fourier_series = [cos.(π * j * (0:T-1) / T .- π/2) for j in 0:n-1]

function expand_in_basis(θ::AbstractVector; basis=fourier_series)
    ## Convert the coefficients to a control vector
    # return # TODO
    return sum(θᵢ * bᵢ for (θᵢ, bᵢ) in zip(θ, basis))
end

In [ ]:
function g(θ)
    a = [expand_in_basis(θ[1:end ÷ 2]); expand_in_basis(θ[end ÷ 2:end])]
    return f(a)
end

∇g(θ) = ForwardDiff.gradient(g, θ)

In [ ]:
## Optimize
# res_fourier = # TODO 

Random.seed!(1234)
res_fourier = optimize(g, ∇g, rand(2n), LBFGS(), inplace=false)

In [ ]:
## Plot the optimized control
a_optimized = [
    reshape(expand_in_basis(res_fourier.minimizer[1:end ÷ 2]), (1,T));
    reshape(expand_in_basis(res_fourier.minimizer[end ÷ 2:end]), (1,T))
]

plot_control(a_optimized, "Fourier series")
